In [69]:
import re
from typing import Optional
from playwright.async_api import async_playwright
import asyncio
import pandas as pd

In [ ]:
async def fetch_full_section_text(url: str) -> str:
    """
    Returns one big string containing:
      • all <div class="Paragraph"> under <span class="SectionBody">
      • followed by the <div class="History"> text (if any)
    """
    async with async_playwright() as pw:
        browser = await pw.chromium.launch(headless=True)
        page    = await browser.new_page()
        await page.goto(url, wait_until="domcontentloaded")

        # 1) Section body text
        await page.wait_for_selector("span.SectionBody")
        paras = await page.locator("span.SectionBody div.Paragraph").all_text_contents()
        body  = "\n\n".join(p.strip() for p in paras if p.strip())

        # 2) History text
        history_locator = page.locator("div.History span.HistoryText")
        if await history_locator.count():
            history = (await history_locator.inner_text()).strip()
        else:
            history = ""

        await browser.close()

    if history:
        return f"{body}\n\n— History —\n\n{history}"
    else:
        return body

def extract_original_enactment_year(history_line: str) -> Optional[int]:
    """
    From a string containing “— History —” followed by amendment entries,
    return the year of the very first chapter (the original enactment year).
    """
    # 1. Split off everything before the History marker
    parts = re.split(r'\n\n—\s*History\s*—\n\n', history_line, flags=re.IGNORECASE)
    if len(parts) < 2:
        # no “— History —” found; nothing to do
        return None
    post_history = parts[1]

    # 2. Find the first "ch. NN-..." or "ch. YYYY-..."
    m = re.search(r'ch\.\s*(\d{2,4})-', post_history)
    if not m:
        return None

    raw = int(m.group(1))
    # Map two-digit years into 1900s or 2000s
    if raw < 100:
        year = 1900 + raw if raw >= 50 else 2000 + raw
    else:
        year = raw

    return year
    
async def extract_florida_statute_links():
    results = []
    async with async_playwright() as p:
        browser = await p.firefox.launch(headless=False)  # Firefox works for this site
        page = await browser.new_page()

        # Navigate to Florida Statutes site
        await page.goto("https://www.flsenate.gov/laws/statutes", wait_until="domcontentloaded")

        # Fill in the search field with 'energy'
        await page.fill("input#filteredData_StatuteSearchQuery", "energy")
        await page.click("input[name='StatutesGoSubmit']")

        # Wait for the results table to load
        await page.wait_for_selector("table.tbl.width100 a", timeout=10000)

        # Extract all links from the first column of the table
        link_elements = await page.query_selector_all("table.tbl.width100 a")
        for link in link_elements:
            href = await link.get_attribute("href")
            text = await link.inner_text()
            if href:
                full_url = f"https://www.flsenate.gov{href}"
                results.append((text.strip(), full_url))

        await browser.close()
        return results

# Jupyter Notebook: Run with
section_links = await extract_florida_statute_links()
print(f"Extracted {len(section_links)} links.")

In [32]:
# for a flat list:
H = list(set(section_links))
df = pd.DataFrame(H, columns=["Statute","Document Link"])
df.to_csv("florida_energy_data_unfiltered.csv", index=False)

In [56]:
texts = []
for i in range(len(section_links)):
    print(i)
    text = await fetch_full_section_text(section_links[i][1])
    texts.append(text)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed


40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304


In [95]:
years = []
for text in texts:
    years.append(extract_original_enactment_year(text))

In [99]:
df = pd.DataFrame({'Document Text':texts, 'year':years})
df['location'] = 'Florida'
df.to_csv("florida_energy_data.csv", index=False)